Main Ref :
https://www.tensorflow.org/probability/examples/TensorFlow_Probability_Case_Study_Covariance_Estimation

In [1]:
%config Completer.use_jedi = False
import tensorflow as tf
import logging

In [2]:
logging.basicConfig(level=logging.DEBUG)

In [3]:
def gen_matrix_from_eigens():
    """
    A = VDV^T
    where V is eigen vector
    D is eigen-values diagonal matrix
    """
    V = tf.constant([[1.,2],[1,3]])
    # print(V)
    D = tf.constant([[-1.,0],[0,7]])
    # print(D)
    A = tf.linalg.matmul(V,tf.linalg.matmul(D,tf.transpose(V)))
    return A

In [4]:
A = gen_matrix_from_eigens()
e,v = tf.linalg.eigh(A)
print(e)


tf.Tensor([-7.858472e-02  8.907858e+01], shape=(2,), dtype=float32)


In [5]:
B = tf.linalg.matmul(v,tf.linalg.matmul(tf.linalg.diag(e),tf.linalg.inv(v)))
B

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[27.      , 41.000004],
       [41.000004, 62.      ]], dtype=float32)>

In [6]:
A

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[27., 41.],
       [41., 62.]], dtype=float32)>

In [7]:
def fix_non_positive_definite_matrix(A,eps=0.01):
    logger = logging.getLogger('fix_non_positive_definite_matrix')
    e,v = tf.linalg.eigh(A)
    e_fix = e * tf.cast(tf.greater(e,0),dtype=tf.float32)+eps *tf.cast(tf.less_equal(e,0),dtype=tf.float32)
    A_fix = tf.matmul(v,tf.matmul(tf.linalg.diag(e_fix),tf.transpose(v)))
    A_norm = tf.norm(A)
    A_fix_norm = tf.norm(A_fix)
    corr_err = tf.math.round(1.0*tf.math.abs(A_norm-A_fix_norm)/A_norm,10)
    logger.info(f'Correction error : {corr_err}')
    return A_fix

In [8]:
A_fixed = fix_non_positive_definite_matrix(A)

INFO:fix_non_positive_definite_matrix:Correction error : 0.0


In [11]:
A

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[27., 41.],
       [41., 62.]], dtype=float32)>

In [9]:
A_fixed

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[27.06168 , 40.959263],
       [40.959263, 62.0269  ]], dtype=float32)>

In [12]:
try:
    tf.linalg.cholesky(A)
except e:
    print(e)

TypeError: catching classes that do not inherit from BaseException is not allowed

In [10]:
tf.linalg.cholesky(A_fixed)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[5.202084  , 0.        ],
       [7.8736258 , 0.18143049]], dtype=float32)>